# Plot clay amounts against major ions

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%run figures.py 'hta_2deg.nc' -c volume


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from cmcrameri import cm
import numpy as np
import copy
import string

file_num=0
fontsize = 24
time = 21

fig, axis = plt.subplots(1,1, figsize=(10,9))

category = 'volume'
plot_vars = ['Clays']

range_sets = [(0,5), (6,9), (10,14)]
labels = [[10, 20, 30, 40, 50, 60],
          [10, 20, 30, 40],
          [1, 10, 20, 30, 40]]

cols = ['tab:blue', 'tab:orange']

linestyles = ['-', '--']
x_label = 'Concentration / mM'
y_label = 'Clay minerals / mols'

names = ['[Mg$\mathbf{^{2+}}$]', '[Ca$\mathbf{^{2+}}$]', '[SO$\mathbf{_4^{2-}}$]']

legend_titles=[1,2,3,4]

#Cell vol /cm^3
cell_vol = 200 * 100**2
#Calcite molar volume cm^3/mol
anhyd_mv = 46.010
vf_to_mols = cell_vol / anhyd_mv

clay_mv = (229.634 + 211.470 + 215.880) / 3  # cm^3/mol ignoring Celadonite as it doesn't precip
amph_mv = (274.660 + 272.920) / 2 # cm^3/mol

scale_factors = {'Clays': cell_vol / clay_mv, 'amph': cell_vol/amph_mv}

base_case_index = [4, 0]

plots.format_axis(axis, font_props)

axis.set_ylabel(y_label, fontweight='bold', fontsize=fontsize)
axis.set_xlabel(x_label, fontweight='bold', fontsize=fontsize)

linestyles = {'Clays': '-', 'amph': '--'}

for i, plot_var in enumerate(plot_vars):
    if plot_var == 'amph':
        names = ['_nolegend_', '_nolegend_']
    iterables = zip(np.arange(len(range_sets)), legend_titles, scale_factors, range_sets, labels, names, cols)
    # Axis level loop
    for j, legend_title, scale, range_set, label_a, name, col in iterables:
        # Line set loop (i.e. groups of lines sharing a property)
        # Line instance loop
        index = np.arange(range_set[0], range_set[1]+1)
        y = []
        x = copy.deepcopy(label_a)
        pop_list = []

        for n, ind in enumerate(index):
            try:
                val = data[category].isel(file_num = ind, time=time, Y=0, Z=0)[plot_var].sum(dim=['X']).to_numpy() * scale_factors[plot_var]
                y.append(val)
            except:
                pop_list.append(n)
                continue

        pop_list.sort(reverse=True)
        for q in pop_list:
            x.pop(q)
        degree_sym = '$\mathbf{\degree}$'
        axis.plot(x, y, label= f'{name}', linestyle=linestyles[plot_var], c=col, linewidth=4)
    axis.legend(title='Seawater component', prop=font_props, title_fontproperties=font_props, framealpha=0, loc='best')

styles = ['-', '--']

fig.tight_layout()

fig.savefig('output/clay_v_cats.png', dpi=300)
fig.show()